# POSTGRES VIA Python

https://www.dataquest.io/blog/loading-data-into-postgres/

### Initialize environment

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\mcsig_thesis\datatransfer\scripts\python.exe -m pip install --upgrade pip' command.


### Connect to postgres

In [1]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

#close the connection
conn.close()
print("Connection closed")

Connection open
Connection closed


### Create table

In [36]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE mensagem_stories")
print ("mensagem_stories dropped")

#Create table mensagem_stories
cur.execute("""
    CREATE TABLE mensagem_stories(
    title text,
    summary text,
    contents text,
    pub_date date,
    spatial text,
    t_begin date,
    t_end date,
    t_type text,
    temporal text,
    link text,
    section text,
    tags text,
    author text,
    publication text
)
""")
conn.commit()
print("mensagem_stories committed")

#close the connection
conn.close()
print("connection closed")

mensagem_stories dropped
mensagem_stories committed
connection closed


## Load data from CSV
Currently setting null date values to 01/01/1900

In [49]:
import psycopg2
import os
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

cwd = os.getcwd()
print("current working directory is ", cwd)

os.chdir("C:\\")
cwd = os.getcwd()
print("current working directory is ", cwd)

path = os.path.join('mcsig_git', 'data_load')
os.chdir(path)
print("Path changed")


#Load table
with open('story2.csv', 'r', encoding='utf-8') as f:
    next(f) #skip header row
    cur.copy_from(f, 'mensagem_stories', sep='|')
conn.commit()
print("data loaded")

cur.execute("""
    ALTER TABLE mensagem_stories ADD id serial PRIMARY KEY, ADD load_id text DEFAULT 'A'
""")

cur.execute("""
    UPDATE mensagem_stories
    set load_id = CONCAT('A',id)
""")
conn.commit()
print("load id assigned")

cur.execute('SELECT title,author,load_id,id FROM mensagem_stories')
all = cur.fetchall()
print(all)

conn.close()
print("connection closed")

Connection open
current working directory is  C:\
current working directory is  C:\
Path changed
data loaded
load id assigned
[('Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir', 'Maria João Martins', 'A1', 1), ('A avó da Uber Eats em Alfama é a salvaçao dos mais velhos', 'Álvaro Filho', 'A2', 2), ('Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia', 'Frederico Raposo ', 'A3', 3), ('"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"', 'Catarina Reis', 'A4', 4), ('Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.', 'Frederico Raposo ', 'A5', 5), ('Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela', 'Álvaro Filho', 'A6', 6), ('Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter', 'Álvaro Filho', 'A7', 7), ('A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se', 'Frederico Raposo ', 'A8', 8)]
connection closed


## Create Relational Database model

In [50]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS story CASCADE")
print ("story dropped")

#Create table story
cur.execute("""
    CREATE TABLE story(
    title text NOT NULL,
    summary text,
    contents text,
    web_link text,
    publish_date date,
    load_id text,
    story_id serial PRIMARY KEY
)
""")
conn.commit()
print("story created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS author CASCADE")
print ("author dropped")

#Create table author
cur.execute("""
    CREATE TABLE author(
    author_name text NOT NULL,
    author_id serial PRIMARY KEY
)
""")
conn.commit()
print("author created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS authorship")
print ("authorship dropped")

#Create table mensagem_stories
cur.execute("""
    CREATE TABLE authorship(
    story_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY (story_id, author_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (author_id) REFERENCES author(author_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("authorship created")

#close the connection
conn.close()
print("connection closed")

story dropped
story created
author dropped
author created
authorship dropped
authorship created
connection closed


## Distribute into database

In [51]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Distribute mensagem stories into stories
cur.execute("""
    INSERT INTO story(title, summary, contents, web_link, publish_date, load_id)
    SELECT title, summary, contents, link, pub_date, load_id
    FROM mensagem_stories
""")
conn.commit()
print("stories loaded")

cur.execute('SELECT title,load_id,story_id FROM story')
all = cur.fetchall()
print(all)

#Distribute mensagem authors into authors
cur.execute("""
    INSERT INTO author(author_name)
    SELECT DISTINCT author
    FROM mensagem_stories
""")
conn.commit()
print("authors loaded")

cur.execute('SELECT * FROM author')
all = cur.fetchall()
print(all)

#close the connection
conn.close()
print("connection closed")

stories loaded
[('Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir', 'A1', 1), ('A avó da Uber Eats em Alfama é a salvaçao dos mais velhos', 'A2', 2), ('Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia', 'A3', 3), ('"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"', 'A4', 4), ('Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.', 'A5', 5), ('Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela', 'A6', 6), ('Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter', 'A7', 7), ('A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se', 'A8', 8)]
authors loaded
[('Álvaro Filho', 1), ('Frederico Raposo ', 2), ('Catarina Reis', 3), ('Maria João Martins', 4)]
connection closed


## Populate authorship

In [52]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

#Relate stories and authors
cur.execute("""
    INSERT INTO authorship(story_id,author_id)
    SELECT mensagem_stories.id, author.author_id
    FROM mensagem_stories
    LEFT JOIN author ON mensagem_stories.author = author.author_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM authorship')
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("Connection closed")

Connection open
[(1, 4), (2, 1), (3, 2), (4, 3), (5, 2), (6, 1), (7, 1), (8, 2)]
Connection closed


## See all articles by author

In [53]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

cur.execute("""
    SELECT author.author_name, story.title
    FROM author
    LEFT JOIN authorship ON author.author_id = authorship.author_id
    INNER JOIN story ON authorship.story_id = story.story_id
    ORDER BY author.author_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("Connection closed")

Connection open
[('Álvaro Filho', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('Álvaro Filho', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('Álvaro Filho', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('Catarina Reis', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('Frederico Raposo ', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se'), ('Frederico Raposo ', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('Frederico Raposo ', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('Maria João Martins', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir')]
Connection closed


## Add Publisher / Publishing

In [54]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS publisher CASCADE")
print ("publisher dropped")


#Create table publisher
cur.execute("""
    CREATE TABLE publisher(
    publisher_name text NOT NULL,
    publisher_site text,
    publisher_description text,
    publisher_id serial PRIMARY KEY
)
""")
conn.commit()
print("publisher created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS publishing")
print ("publishing dropped")

#Create table publishing
cur.execute("""
    CREATE TABLE publishing(
    story_id INT NOT NULL,
    publisher_id INT NOT NULL,
    PRIMARY KEY (story_id, publisher_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (publisher_id) REFERENCES publisher(publisher_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("publishing created")

#Distribute publishers
cur.execute("""
    INSERT INTO publisher(publisher_name)
    SELECT DISTINCT publication
    FROM mensagem_stories
""")
conn.commit()
print("publishers loaded")

cur.execute('SELECT * FROM publisher')
all = cur.fetchall()
print(all)

#Relate stories and authors
cur.execute("""
    INSERT INTO publishing(story_id,publisher_id)
    SELECT mensagem_stories.id, publisher.publisher_id
    FROM mensagem_stories
    LEFT JOIN publisher ON mensagem_stories.publication = publisher.publisher_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM publishing')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT publisher.publisher_name, story.title
    FROM publisher
    LEFT JOIN publishing ON publisher.publisher_id = publishing.publisher_id
    INNER JOIN story ON publishing.story_id = story.story_id
    ORDER BY publisher.publisher_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

publisher dropped
publisher created
publishing dropped
publishing created
publishers loaded
[('mensagem', None, None, 1)]
[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[('mensagem', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir'), ('mensagem', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('mensagem', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('mensagem', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('mensagem', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('mensagem', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('mensagem', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('mensagem', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se')]
connection closed


## Add Section / Sectioning

In [55]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS section CASCADE")
print ("section dropped")


#Create table publisher
cur.execute("""
    CREATE TABLE section(
    section_name text NOT NULL,
    section_id serial PRIMARY KEY
)
""")
conn.commit()
print("section created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS sectioning")
print ("sectioning dropped")

#Create table publishing
cur.execute("""
    CREATE TABLE sectioning(
    story_id INT NOT NULL,
    section_id INT NOT NULL,
    PRIMARY KEY (story_id, section_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (section_id) REFERENCES section(section_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("sectioning created")

#Distribute publishers
cur.execute("""
    INSERT INTO section(section_name)
    SELECT DISTINCT section
    FROM mensagem_stories
""")
conn.commit()
print("sections loaded")

cur.execute('SELECT * FROM section')
all = cur.fetchall()
print(all)

#Relate stories and sections
cur.execute("""
    INSERT INTO sectioning(story_id,section_id)
    SELECT mensagem_stories.id, section.section_id
    FROM mensagem_stories
    LEFT JOIN section ON mensagem_stories.section = section.section_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM sectioning')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT section.section_name, story.title
    FROM section
    LEFT JOIN sectioning ON section.section_id = sectioning.section_id
    INNER JOIN story ON sectioning.story_id = story.story_id
    ORDER BY section.section_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

section dropped
section created
sectioning dropped
sectioning created
sections loaded
[('cidade', 1), ('bairros', 2)]
[(1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 1)]
[('bairros', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir'), ('bairros', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('bairros', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('bairros', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('bairros', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('bairros', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('bairros', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('cidade', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se')]
connection closed


## Theme / Theming (Tags)

In [56]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS theme CASCADE")
print ("theme dropped")


#Create table theme
cur.execute("""
    CREATE TABLE theme(
    theme_name text NOT NULL,
    theme_id serial PRIMARY KEY
)
""")
conn.commit()
print("theme created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS stheming")
cur.execute("DROP TABLE IF EXISTS theming")
print ("theming dropped")

#Create table theming
cur.execute("""
    CREATE TABLE theming(
    story_id INT NOT NULL,
    theme_id INT NOT NULL,
    PRIMARY KEY (story_id, theme_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (theme_id) REFERENCES theme(theme_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("theming created")

#Distribute publishers
cur.execute("""
    INSERT INTO theme(theme_name)
    SELECT DISTINCT tags
    FROM mensagem_stories
""")
conn.commit()
print("themes loaded")

cur.execute('SELECT * FROM theme')
all = cur.fetchall()
print(all)

#Relate stories and themes
cur.execute("""
    INSERT INTO theming(story_id,theme_id)
    SELECT mensagem_stories.id, theme.theme_id
    FROM mensagem_stories
    LEFT JOIN theme ON mensagem_stories.tags = theme.theme_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM theming')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT theme.theme_name, story.title
    FROM theme
    LEFT JOIN theming ON theme.theme_id = theming.theme_id
    INNER JOIN story ON theming.story_id = story.story_id
    ORDER BY theme.theme_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

theme dropped
theme created
theming dropped
theming created
themes loaded
[('', 1)]
[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[('', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir'), ('', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se')]
connection closed


## Employment

In [57]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS employment CASCADE")
print ("employment dropped")


#Create table employment
cur.execute("""
    CREATE TABLE employment(
    publisher_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY (publisher_id, author_id),
    FOREIGN KEY (publisher_id) REFERENCES publisher(publisher_id) ON UPDATE CASCADE,
    FOREIGN KEY (author_id) REFERENCES author(author_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("employment created")


#Relate authors and publishers
cur.execute("""
    INSERT INTO employment(publisher_id,author_id)
    SELECT DISTINCT publisher.publisher_id, author.author_id
    FROM mensagem_stories
    LEFT JOIN author ON mensagem_stories.author = author.author_name
    LEFT JOIN publisher ON mensagem_stories.publication = publisher.publisher_name
    ORDER BY author.author_id
""")
conn.commit()
cur.execute('SELECT * FROM employment')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT author.author_name, publisher.publisher_name
    FROM author
    LEFT JOIN employment ON author.author_id = employment.author_id
    INNER JOIN publisher ON employment.publisher_id = publisher.publisher_id
    ORDER BY author.author_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

employment dropped
employment created
[(1, 1), (1, 2), (1, 3), (1, 4)]
[('Álvaro Filho', 'mensagem'), ('Catarina Reis', 'mensagem'), ('Frederico Raposo ', 'mensagem'), ('Maria João Martins', 'mensagem')]
connection closed


## Story Relations

In [58]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS storyref CASCADE")
print ("storyref dropped")


#Create table storyref
cur.execute("""
    CREATE TABLE storyref(
    storya_id INT NOT NULL,
    storyb_id INT NOT NULL,
    PRIMARY KEY (storya_id, storyb_id),
    FOREIGN KEY (storya_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (storyb_id) REFERENCES story(story_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("storyref created")

print("storyref empty")


#close the connection
conn.close()
print("connection closed")

storyref dropped
storyref created
storyref empty
connection closed


## Story typing

In [59]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS stype CASCADE")
print ("stype dropped")


#Create table theme
cur.execute("""
    CREATE TABLE stype(
    stype_name text NOT NULL,
    stype_description text,
    stype_id serial PRIMARY KEY
)
""")
conn.commit()
print("stype created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS styping")
print ("styping dropped")

#Create table theming
cur.execute("""
    CREATE TABLE styping(
    story_id INT NOT NULL,
    stype_id INT NOT NULL,
    PRIMARY KEY (story_id, stype_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (stype_id) REFERENCES stype(stype_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("styping created")

print("styping empty")

#close the connection
conn.close()
print("connection closed")

stype dropped
stype created
styping dropped
styping created
styping empty
connection closed


## Instance

In [12]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS user_gazetteer CASCADE")
cur.execute("DROP TABLE IF EXISTS user_gazetteer_point CASCADE")
print ("User Gazetteer dropped")

#Create table sistance. 
cur.execute("""
    CREATE TABLE user_gazetteer(
    place_name TEXT,
    place_desc TEXT,
    geom GEOMETRY,
    place_id serial PRIMARY KEY
)
""")
conn.commit()
print("user_gazetteer created")

#Distribute places
cur.execute("""
    INSERT INTO user_gazetteer(place_desc)
    SELECT spatial
    FROM mensagem_stories
""")
conn.commit()
print("instances pre-loaded")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS sinstance CASCADE")
print ("sinstance dropped")

#Create table sistance. 
cur.execute("""
    CREATE TABLE sinstance(
    story_id INT NOT NULL,
    t_begin DATE,
    t_end DATE,
    t_type text,
    t_desc text,
    place_id INT,
    sinstance_id serial PRIMARY KEY,
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (place_id) REFERENCES user_gazetteer(place_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("sinstance created")

#Distribute instances
cur.execute("""
    INSERT INTO sinstance(story_id, t_begin, t_end, t_type, t_desc, place_id)
    SELECT ms.id, ms.t_begin, ms.t_end, ms.t_type, ms.temporal, ug.place_id
    FROM mensagem_stories AS ms
    LEFT JOIN user_gazetteer AS ug ON ms.spatial = ug.place_desc
    LEFT JOIN story ON ms.load_id = story.load_id
    ORDER BY ms.id
""")
conn.commit()
print("instances pre-loaded")

cur.execute('SELECT * FROM sinstance')
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

User Gazetteer dropped
user_gazetteer created
instances pre-loaded
sinstance dropped
sinstance created
instances pre-loaded
[(1, datetime.date(1900, 1, 1), datetime.date(1900, 1, 1), 'na', '', 1, 1), (2, datetime.date(2020, 3, 1), datetime.date(1900, 1, 1), 'ongoing', 'COVID', 2, 2), (3, datetime.date(2020, 3, 1), datetime.date(1900, 1, 1), 'ongoing', 'COVID', 3, 3), (4, datetime.date(2016, 1, 1), datetime.date(1900, 1, 1), 'ongoing', '2016-present', 4, 4), (5, datetime.date(2021, 1, 19), datetime.date(2021, 1, 19), 'concrete', '19-Jan', 5, 5), (6, datetime.date(2020, 3, 1), datetime.date(1900, 1, 1), 'ongoing', 'March 2020 - current', 6, 6), (7, datetime.date(2020, 2, 1), datetime.date(1900, 1, 1), 'ongoing', 'Feb 2020 - current', 7, 7), (8, datetime.date(2021, 3, 5), datetime.date(2021, 10, 5), 'predicted', '3 March 2021 to 5 October 2021', 8, 8)]
connection closed


## Next:

1. Integrate with GIT - DONE
2. Finish adding the full data model - DONE
3. Dimension the data to the model - DONE